In [83]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import api

import importlib
importlib.reload(api)
import urllib.request
import librosa
import os

import pandas as pd

cid = '5c3d73b1e0734983aec11219f14fef4f'
secret = 'f23dddb03f9a4097887e4a251758f86d'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [6]:
bay_area = sp.search(q = 'genre:Bay%area%hip%hop', type='artist', limit = 50)

chinese = sp.search(q = 'genre:chinese%hip%hop', type='artist', limit = 50)


In [7]:
bay_area_artists = []
for i in bay_area['artists']['items']:
    bay_area_artists.append(i)
    
chinese_artists = []
for i in chinese['artists']['items']:
    chinese_artists.append(i)
    

In [77]:
for artist in bay_area_artists[10:15]:
    api.show_artist(artist)
    print('\n')

Name  Lil Los
Popularity: 33
Genres: ['bay area hip hop']


Name  Casper Locs
Popularity: 31
Genres: ['bay area hip hop']


Name  Big Rome
Popularity: 31
Genres: ['bay area hip hop']


Name  G-Stack
Popularity: 32
Genres: ['bay area hip hop']


Name  Conspiracy
Popularity: 30
Genres: ['bay area hip hop']




In [76]:
for artist in chinese_artists[:5]:
    api.show_artist(artist)
    print('\n')

Name  Higher Brothers
Popularity: 67
Genres: ['chinese hip hop']


Name  Kris Wu
Popularity: 59
Genres: ['chinese hip hop', 'chinese idol pop']


Name  Jony J
Popularity: 48
Genres: ['chinese hip hop']


Name  VaVa
Popularity: 46
Genres: ['chinese hip hop']


Name  ØZI
Popularity: 54
Genres: ['chinese hip hop', 'taiwan hip hop']




In [71]:
artists_top_tracks = {}
artists_top_tracks['bay_area'] = []
artists_top_tracks['chinese'] = []



In [ ]:
## Getting bay area artist tracks

In [72]:
for artist in bay_area_artists[:1]:
  
    artist_album_dic = {}
    artist_album_dic[artist['name']] = []
    
    ##gets top tracks for artist 
    
    artist_top_dic = {}
    artist_top_dic['tracks'] = []
    
    top_tracks = api.top_songs(artist)
    
    for i in top_tracks:
        artist_top_dic['name'] = artist['name']
        artist_top_dic['tracks'].append(i)
    
    artists_top_tracks['bay_area'].append(artist_top_dic)
    


In [ ]:
## Getting chinese artist tracks

In [73]:
##gets all albums
for artist in chinese_artists[:1]:
  
    artist_album_dic = {}
    artist_album_dic[artist['name']] = []
    
    ##gets top tracks for artist 
    
    artist_top_dic = {}
    artist_top_dic['tracks'] = []
    
    top_tracks = api.top_songs(artist)
    
    for i in top_tracks:
        artist_top_dic['name'] = artist['name']
        artist_top_dic['tracks'].append(i)
    
    artists_top_tracks['chinese'].append(artist_top_dic)
    



In [ ]:
## download songs for bay area artists 

In [28]:
for i in artists_top_tracks['bay_area']:
    directory =  '../test-music-data'
    cat = 'bay_area'
    artist_name = i['name']
    top_tracks = i['tracks']
    for i in top_tracks:
            api.download_track(i,directory,\
                               cat, artist_name)


189459

In [75]:
## download songs for chinese artists 

In [74]:
for i in artists_top_tracks['chinese']:
    directory =  '../test-music-data'
    cat = 'chinese'
    artist_name = i['name']
    top_tracks = i['tracks']
    for i in top_tracks:
            api.download_track(i,directory,\
                               cat, artist_name)

No Preview URL found for track - Made In China


In [89]:
table_dic = []

for loc in artists_top_tracks:
    for artist in artists_top_tracks[loc]:
        for tracks in artist['tracks']:
            dic = {}
            dic['type'] = loc
            dic['name'] = artist['name']
            dic['track_name'] = tracks['name']
            dic['uri'] = tracks['uri']
            table_dic.append(dic)
    

pd.DataFrame(table_dic)

,type,name,track_name,uri
0,bay_area,Caleborate,4 Willem,spotify:track:4nWPmn1tqegITYKdZOJpKv
1,bay_area,Caleborate,Caught Up,spotify:track:4jvQ3EvoTkgeXiqTjLlhPv
2,bay_area,Caleborate,Clicquot Shower - A COLORS SHOW,spotify:track:3iBBYvj3I5DNSuUgFjushj
3,bay_area,Caleborate,El Portal,spotify:track:7fWMxQKbY4FxofuYz1ZMDx
4,bay_area,Caleborate,Pull Up,spotify:track:7jNKltvrrClDaaG81flM1w
5,bay_area,Caleborate,Untitled (Hit Record),spotify:track:3mM3yqzLY8k5kpVawTqSsz
6,bay_area,Caleborate,home,spotify:track:7kaIJuX8fxDgTFumxFmCWi
7,bay_area,Caleborate,These Days,spotify:track:1c0IND0G0SlApPtxfbhlEZ
8,bay_area,Caleborate,Away From,spotify:track:3SVeFVQS1H0XRLjkEccPKC
9,bay_area,Caleborate,Bankrobber,spotify:track:3SqVtIMesHETZDm3UU2Pb4


In [90]:
features = sp.audio_features(all_tracks[0]['uri'])
features

[{'danceability': 0.792,
  'energy': 0.426,
  'key': 2,
  'loudness': -12.432,
  'mode': 0,
  'speechiness': 0.324,
  'acousticness': 0.74,
  'instrumentalness': 1.01e-06,
  'liveness': 0.0883,
  'valence': 0.693,
  'tempo': 82.561,
  'type': 'audio_features',
  'id': '6IkTlI62akMNoUxfxHNRZH',
  'uri': 'spotify:track:6IkTlI62akMNoUxfxHNRZH',
  'track_href': 'https://api.spotify.com/v1/tracks/6IkTlI62akMNoUxfxHNRZH',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6IkTlI62akMNoUxfxHNRZH',
  'duration_ms': 189460,
  'time_signature': 4}]

In [47]:
playlist_id = 'spotify:user:spotifycharts:playlist:37i9dQZF1DX9sQDbOMReFI'
results = sp.playlist(playlist_id)

In [48]:
results['tracks']['items'][0]['track']['name']

'Nuthin\' But A "G" Thang'